In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import plotly
from plotly import graph_objects as go

In [49]:
df = pd.read_csv('../Data/BitcoinHeistData.csv')

In [50]:
df

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky
...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2018,330,0,0.111111,1,0,1,1.255809e+09,white
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2018,330,0,1.000000,1,0,1,4.409699e+07,white
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2018,330,2,12.000000,6,6,35,2.398267e+09,white
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2018,330,0,0.500000,1,0,1,1.780427e+08,white


In [51]:
# Plot Year bar graph
year_dist = df['year'].value_counts()
year_dist = year_dist.sort_index()
fig = go.Figure(data=[go.Bar(x=year_dist.index, y=year_dist.values)])
fig.update_layout(title_text='Year Distribution')
fig.show()

In [52]:
# Plot Day distribution
day_dist = df['day'].value_counts()
day_dist = day_dist.sort_index()
fig = go.Figure(data=[go.Bar(x=day_dist.index, y=day_dist.values)])
fig.update_layout(title_text='Day Distribution')
fig.show()

In [53]:
def is_leap_year(year):
    # Function to check if a year is a leap year
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        return True
    return False

cache = {}

def day_number_to_date(day_number, year):
    if (day_number, year) in cache:
        return cache[(day_number, year)]
    
    days_in_month = [31, 28 if not is_leap_year(year) else 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    if day_number < 1 or day_number > sum(days_in_month):
        return None  # Invalid day number
    
    month = 1
    while day_number > days_in_month[month - 1]:
        day_number -= days_in_month[month - 1]
        month += 1
    cache[(day_number, year)] = (month, day_number)
    return month, day_number

# Example usage:
day = 200
year = 2023  # Replace with your desired year
result = day_number_to_date(day, year)

if result:
    month, date = result
    print(f"Day {day} in {year} is in month {month} and the date is {date}.")
else:
    print("Invalid day number.")

Day 200 in 2023 is in month 7 and the date is 19.


In [54]:
df['month'] = df.apply(lambda row: day_number_to_date(row['day'], row['year'])[0], axis=1)
df['date'] = df.apply(lambda row: day_number_to_date(row['day'], row['year'])[1], axis=1)

In [55]:
df

,address,year,day,length,weight,count,looped,neighbors,income,label,month,date
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber,1,11
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky,5,11
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber,9,2
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber,11,17
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky,8,25
...,...,...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2018,330,0,0.111111,1,0,1,1.255809e+09,white,11,26
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2018,330,0,1.000000,1,0,1,4.409699e+07,white,11,26
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2018,330,2,12.000000,6,6,35,2.398267e+09,white,11,26
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2018,330,0,0.500000,1,0,1,1.780427e+08,white,11,26


In [56]:
# Plot Month distribution
month_dist = df['month'].value_counts()
month_dist = month_dist.sort_index()
fig = go.Figure(data=[go.Bar(x=month_dist.index, y=month_dist.values)])
fig.update_layout(title_text='Month Distribution')
# Change x-axis labels
fig.update_xaxes(ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
                           'Nov', 'Dec'])
fig.show()

In [57]:
def plot_monthly_distribution(month):
    # Plot the distribution of the month
    month_df = df[df['month'] == month]
    day_dist = month_df['date'].value_counts()
    day_dist = day_dist.sort_index()
    fig = go.Figure(data=[go.Bar(x=day_dist.index, y=day_dist.values)])
    fig.update_layout(title_text=f'Day Distribution for Month {month}')
    return fig

In [58]:
# Plot Date distribution for each month
for i in range(1, 13):
    fig = plot_monthly_distribution(i)
    fig.show()

In [59]:
def get_range_per_month(month_num, year):
    isLeap = is_leap_year(year)
    if month_num in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    elif month_num == 2:
        if isLeap:
            return 29
        else:
            return 28
    else:
        return 30    

In [60]:
def plot_month_dist_for_year(year, month):
    # Plot the distribution of the date
    year_df = df[df['year'] == year]
    month_df = year_df[year_df['month'] == month]
    day_dist = month_df['date'].value_counts()
    day_dist = day_dist.sort_index()
    fig = go.Figure(data=[go.Bar(x=day_dist.index, y=day_dist.values)])
    month_name = datetime.date(1900, month, 1).strftime('%B')
    fig.update_layout(title_text=f'Day Distribution for {month_name} {year}')
    number_of_days = get_range_per_month(month, year)
    fig.update_xaxes(range=[1, number_of_days])
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Number of Attacks')
    fig.update_xaxes(tickmode='linear')
    return fig

In [61]:
# Plot Date distribution for each month in each year
min_year = df['year'].min()
max_year = df['year'].max()
for i in range(min_year, max_year + 1):
    ydf = df[df['year'] == i]
    for j in range(1, 13):
        fig = plot_month_dist_for_year(i, j)
        fig.show()